<a href="https://colab.research.google.com/github/isichan0501/BotHelper/blob/main/japanese_alpaca_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#文字コードエラーでcolobが動かない時に
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
%cd "/content/drive/My Drive/"

#check exist data
import os
if not os.path.exists("alpaca_llama_index"):
  !git clone https://github.com/isichan0501/alpaca_llama_index.git
%cd "alpaca_llama_index"


# if not os.path.exists("japanese-alpaca-lora"):
#   !git clone https://github.com/masa3141/japanese-alpaca-lora.git
# train_data_filepath = "japanese-alpaca-lora/data/japanese_alpaca_data.json"

In [ ]:
# パッケージのインストール
!pip install llama-index
!pip install -q bitsandbytes datasets accelerate loralib sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git@main 
!pip install -q git+https://github.com/huggingface/peft.git


In [ ]:
import torch
from langchain.llms.base import LLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding
from llama_index import SimpleDirectoryReader, LLMPredictor, PromptHelper, GPTSimpleVectorIndex
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LLaMAForCausalLM, LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, get_peft_model_state_dict

hf_model_path = "decapoda-research/llama-7b-hf"
alpaca_model_path = "data/japanese-lora-alpaca"

# run llama-index script



In [ ]:
#alpaca-japaneseをダウンロード
# Import the necessary libraries
import requests
import zipfile
import io

# Enter the shared link for the folder you want to download
shared_link = 'https://drive.google.com/drive/folders/1e7Qel8_fSwvpC1utoyGwAItgpcKBL_90?usp=share_link'

# Get the ID of the folder from the shared link
folder_id = shared_link.split('/')[-2]

# Create a URL to download the folder as a zip file
url = f'https://drive.google.com/uc?id={folder_id}&export=download'

# Send a request to download the folder as a zip file
r = requests.get(url)

# Unzip the downloaded file
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(alpaca_model_path)


In [ ]:


tokenizer = LLaMATokenizer.from_pretrained(hf_model_path)

model = LLaMAForCausalLM.from_pretrained(
    hf_model_path,
    load_in_8bit=True, #Dissabling could solve some errors
    device_map="auto",
)
model = PeftModel.from_pretrained(model, alpaca_model_path)


device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
max_length = 1500 #2048
max_new_tokens = 48


class LLaMALLM(LLM):
    def _call(self, prompt, stop=None):
        prompt += "### Response:"

        inputs = tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].cuda()
        
        generation_config = GenerationConfig(
            temperature=0.6,
            top_p=0.95,
            repetition_penalty=1.15,
        )
        with torch.no_grad():
            generation_output = model.generate(
                input_ids=input_ids,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=128,
            )
        response = ""
        for s in generation_output.sequences:
            response += tokenizer.decode(s)
            
        response = response[len(prompt):]
        print("Model Response:", response)
        return response

    def _identifying_params(self):
        return {"name_of_model": "alpaca"}

    def _llm_type(self):
        return "custom"

max_input_size = max_length
num_output = max_new_tokens
max_chunk_overlap = 20

In [ ]:
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
documents = SimpleDirectoryReader('data').load_data()
llm_predictor = LLMPredictor(llm=LLaMALLM())
index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper)



In [ ]:
index.save_to_disk('index.json')
new_index = GPTSimpleVectorIndex.load_from_disk('index.json', embed_model=embed_model, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

response = new_index.query("ぼっちちゃんの得意な楽器は？")
print(response.response)

response = new_index.query("ぼっちちゃんについてどう思う？")
print(response.response)

response = new_index.query("喜多ちゃんの一番中の良い友達は？")
print(response.response)

In [ ]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token


In [ ]:
# Please upload japanese_alpaca_data.json
data = load_dataset("json", data_files="japanese_alpaca_data.json")

In [ ]:
# Split data set into train and validation data.
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]


In [ ]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""


def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x)))


In [ ]:
world_size = int(os.environ.get('WORLD_SIZE', 1))
ddp = world_size != 1

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        save_steps=200,
        output_dir="japanese-lora-alpaca",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False


In [ ]:
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))


In [ ]:
# Start finetunig. It takes around 7 hours on Google Colab PRO+.
trainer.train()


In [ ]:
model.save_pretrained("japanese-lora-alpaca")

## Load finetuned model and check the result

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

In [ ]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model_custom = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model_original = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model_custom = PeftModel.from_pretrained(model_custom, "japanese-lora-alpaca")
model_original = PeftModel.from_pretrained(model_original, "tloen/alpaca-lora-7b")

In [ ]:
def generate_instruction_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [ ]:
evaluate(model_custom,"日本の首都はどこですか？")

In [ ]:
evaluate(model_original,"日本の首都はどこですか？")